## Basic Query with the LangChain Wrapper

In [15]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-4")
llm("explain large language models in one sentence")

/Users/sanskar/git/work/hot-track-langchain/venv/lib/python3.10/site-packages/langchain_community/llms/openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/Users/sanskar/git/work/hot-track-langchain/venv/lib/python3.10/site-packages/langchain_community/llms/openai.py:1061: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


## Importing and defining Schema for the chat messages

In [10]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [11]:
chat = ChatOpenAI(model_name="gpt-4",temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
response=chat(messages)

print(response.content,end='\n')

AttributeError: module 'openai' has no attribute 'error'

## Import prompt and define PromptTemplate

In [ ]:
from langchain import PromptTemplate

template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

In [ ]:
llm(prompt.format(concept="autoencoder"))

'Autoencoder is a type of artificial neural network used for learning efficient codings of input data. It consists of two parts: an encoder, which compresses the input data into a latent-space representation, and a decoder, which reconstructs the original data from the latent representation. They are primarily used for dimensionality reduction or noise reduction purposes.'

## Importing and defining the LLMChain with language model and prompt as arguments.

In [ ]:

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("autoencoder"))

An autoencoder is a type of artificial neural network used for learning efficient codings of input data. It is an unsupervised learning technique where the network is trained to attempt to copy its input to its output, using a hidden layer that describes the code to represent the input. Essentially, autoencoders are used for reducing the dimensionality of data (compression) and noise reduction.


### Second Prompt

In [ ]:
second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

## Define a sequential chain using the two chains above


In [ ]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("self-attention")
print(explanation)



> Entering new SimpleSequentialChain chain...
Self-attention, in the context of deep learning models, is a mechanism that allows the model to focus on different parts of the input sequence when producing an output. It computes a weighted average of input features where weights are determined by the interactions between inputs themselves. This helps in contextualizing the information in the data, particularly useful in natural language processing tasks.
Imagine you're reading a book of your favorite superhero story. You understand what's happening not only by reading each sentence or each word separately but because you're relating it with previous sentences, the whole page or even previous chapters. 

For example, when you read the word "he," you often know who this "he" is, because of what you read before or what you guess will happen. This is called context: the "he" does not mean the same thing all the time, it depends on the situation. Similarly, self-attention in the world of sm

In [ ]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([explanation])

In [ ]:

# Individual text chunks can be accessed with "page_content"

texts[0].page_content


"Imagine you're reading a book of your favorite superhero story. You understand what's happening not"

## Import and instantiate OpenAI embeddings

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model_name="text-embedding-ada-002")

/Users/sanskar/git/work/langidk/langchain/lib/python3.9/site-packages/langchain/embeddings/openai.py:217: UserWarning: WARNING! model_name is not default parameter.
                    model_name was transferred to model_kwargs.
                    Please confirm that model_name is what you intended.
  warnings.warn(


In [ ]:

# Turn the first text chunk into a vector with the embedding

query_result = embeddings.embed_query(texts[0].page_content)
print(query_result)

[-0.021061465044486107, -0.014878282253069653, 0.002943222397048378, 0.009964860928584621, -0.002211729752608917, 0.016424078416585053, -0.007204511168777163, -0.055068965740663764, -0.013084054862628676, -0.010199489955019715, 0.03941778447451384, -0.005151501832365317, -0.009136755907510126, -0.00145263375492638, 0.038065213529929795, 0.006956079941851585, 0.017845656638331236, 0.016920940870436205, -0.003409031296779799, -0.01324967568057906, 0.005051439138438304, 0.026319930205362392, 0.00815683133765316, -0.012442273960240292, 0.017666235482535507, -0.007011286881168381, 0.005096294427387236, -0.05639393228426685, -0.02114427452213873, 0.0034073060217184887, 0.012683804086920627, 0.006221136980120146, 0.0163826718151136, -0.0313575676089715, -0.04297863671705997, 0.0011904005603409614, -0.009516303382482449, -0.025767858949549305, 0.015030100870529554, -0.0163826718151136, 0.008419064765069222, -0.0207302215459402, 0.01287702930585198, -0.03958340715510937, -0.017058957287405623, 

## Creating an agent to solve a simple quadratic equation

In [ ]:
# Import Python REPL tool and instantiate Python agent

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

agent_executor = create_python_agent(
    llm=OpenAI(model_name="gpt-3.5-turbo-1106" ,temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)


In [ ]:
# Execute the Python agent

agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x -1")     



> Entering new AgentExecutor chain...
I can use the quadratic formula to find the roots of the quadratic function.
Action: Python_REPL
Action Input: import cmath
Observation: 
Thought:I need to define the coefficients of the quadratic function and then use the quadratic formula to find the roots.
Action: Python_REPL
Action Input: a = 3, b = 2, c = -1
Observation: SyntaxError('cannot assign to literal', ('<string>', 1, 5, 'a = 3, b = 2, c = -1\n'))
Thought:I need to define the coefficients separately.
Action: Python_REPL
Action Input: a = 3
Observation: 
Thought:I need to define the other coefficients separately as well.
Action: Python_REPL
Action Input: b = 2
Observation: 
Thought:I can now use the quadratic formula to find the roots.
Action: Python_REPL
Action Input: root1 = (-b + cmath.sqrt(b**2 - 4*a*c)) / (2*a), root2 = (-b - cmath.sqrt(b**2 - 4*a*c)) / (2*a)
Observation: SyntaxError('cannot assign to operator', ('<string>', 1, 9, 'root1 = (-b + cmath.sqrt(b**2 - 4*a*c)) / (2*a),

'root1 = (0.3333333333333333+0j), root2 = (-1+0j)'